# Example 4: Console Logging - Real-Time Output

## What You'll Learn

This notebook demonstrates console logging with handlers:

- **ConsoleHandler** - Output logs to console
- **Console_ServiceConfig** - Configure console output
- **Output Modes** - JSON vs Text formatting
- **Logger with Handler** - Complete logging setup
- **Real-World Examples** - Development and debugging scenarios


## Setup - Imports


In [ ]:
import asyncio

from dc_logger.client.base import HandlerBufferSettings, HandlerInstance, Logger
from dc_logger.client.models import LogEntry, LogLevel
from dc_logger.services.console.base import ConsoleServiceConfig, ConsoleHandler

print("All imports successful!")


---
## Part 1: Basic Console Logging - Text Mode


In [ ]:
print("=" * 70)
print("Console Logging - Text Mode")
print("=" * 70)

# Step 1: Create Console Service Config
console_config = ConsoleServiceConfig(
    output_mode="console",
    output_type="text"  # "text" or "json"
)

# Step 2: Create Console Handler with buffer settings
buffer_settings = HandlerBufferSettings()
console_handler = ConsoleHandler(
    buffer_settings=buffer_settings,
    service_config=console_config
)

# Step 3: Create HandlerInstance (concrete class that wraps ServiceHandler)
# HandlerInstance adds filtering logic (log_level and log_method)
handler_instance = HandlerInstance(
    service_handler=console_handler,
    handler_name="console_text",
    log_level=LogLevel.INFO,  # Will be updated by Logger.__post_init__
    log_method=["POST", "PUT", "DELETE", "PATCH", "COMMENT", "GET"]
)

# Step 4: Create Logger with the handler
logger = Logger(
    app_name="console_demo_logger",
    handlers=[handler_instance],
    show_debugging=False  # False = no DEBUG messages, True = show DEBUG messages
)

# Simple log
print("\nSimple INFO log:")
await logger.log(LogLevel.INFO, "Application started successfully")

# Adding user context to the log
print("\n\nLog with user context:")
await logger.log(LogLevel.INFO, "User logged in", user="alice@example.com")


# Log with action
print("\n\nLog with action:")
await logger.log(LogLevel.INFO, "Database query executed", action="db_query", duration_ms=45)

print("\n\nLog with action and extra fields:")
#you can add any other field to the log
await logger.log(LogLevel.INFO, "Database query executed", action="db_query", duration_ms=45, extra={"user": "alice@example.com"}, method="GET",id = "1234567890")



---
## Part 2: Console Logging - JSON Mode


In [ ]:
print("=" * 70)
print("Console Logging - JSON Mode")
print("=" * 70)

# Create console handler with JSON output
console_json_config = ConsoleServiceConfig(
    output_mode="console",
    output_type="json"  # JSON format for rich logs
)

buffer_settings_json = HandlerBufferSettings()
console_json_handler = ConsoleHandler(
    buffer_settings=buffer_settings_json,
    service_config=console_json_config
)

# Create handler instance
json_handler_instance = HandlerInstance(
    service_handler=console_json_handler,
    handler_name="console_json",
    log_level=LogLevel.INFO  # Will be updated by Logger.__post_init__
)

# Create logger with JSON console handler
json_logger = Logger(
    app_name="console_json_demo",
    handlers=[json_handler_instance],
    show_debugging=False  # False = no DEBUG messages, True = show DEBUG messages
)

# Log with rich context
print("\nRich log with entity:")
entry = logger.create_entry(
    level=LogLevel.INFO,
    message="Dataset processed",
    entity={
        "type": "dataset",
        "id": "ds_sales_2024",
        "name": "Sales Data 2024"
    },
    action="process_dataset",
    duration_ms=1234,
    extra={"rows_processed": 5000, "errors": 0}
)
await json_logger.write(entry)


---
## Part 3: Different Log Levels


In [ ]:
print("=" * 70)
print("Different Log Levels")
print("=" * 70)
print("Note: DEBUG is controlled by show_debugging=True/False")
print("INFO/WARNING/ERROR/CRITICAL always show")
print("=" * 70)

# Using text logger for clarity
print("\nDEBUG:")
await logger.debug("Checking configuration files",)


print("\n\nINFO:")
await logger.info("Service initialized",)


print("\n\nWARNING:")
await logger.warning("Cache size approaching limit", extra={"cache_usage": "85%"})


print("\n\nERROR:")
await logger.error("Failed to connect to database", extra={"retry_attempt": 3})

print("\n\nCRITICAL:")

await logger.critical("System out of memory", extra={"memory_used": "98%"})

print("\n\n")


---
## Part 4: HTTP Request Logging


In [ ]:
print("=" * 70)
print("HTTP Request Logging")
print("=" * 70)

# Log API request (JSON mode for full detail)
print("\nAPI Request Log:")
entry_api_success = LogEntry.create(
    level=LogLevel.INFO,
    message="API request processed",
    app_name="console_json_demo",
    http_details={
        "method": "POST",
        "url": "/api/users",
        "status_code": 201,
        "headers": {"Content-Type": "application/json"},
        "params": {"expand": "profile"}
    },
    duration_ms=156
)
await json_logger.write(entry_api_success)

# Log failed API request
print("\n\nFailed API Request:")
entry_api_error = LogEntry.create(
    level=LogLevel.ERROR,
    message="API request failed",
    app_name="console_json_demo",
    http_details={
        "method": "GET",
        "url": "/api/orders/999",
        "status_code": 404
    },
    duration_ms=89,
    extra={"error": "Order not found"}
)
await json_logger.write(entry_api_error)


---
## Part 5: Multi-Tenant Context


In [ ]:
print("=" * 70)
print("Multi-Tenant Context Logging")
print("=" * 70)

# Log with multi-tenant context
print("\nSaaS Platform Log:")
entry_tenant = LogEntry.create(
    level=LogLevel.INFO,
    message="Project created",
    app_name="console_json_demo",
    user_id="user_alice_123",
    tenant_id="tenant_acme_corp",
    organization_id="org_enterprise",
    session_id="sess_web_xyz789",
    entity={
        "type": "project",
        "id": "proj_website_redesign",
        "name": "Website Redesign Project"
    },
    action="create_project"
)
await json_logger.write(entry_tenant)

# Log user action
print("\n\nUser Activity Log:")
entry_user = LogEntry.create(
    level=LogLevel.INFO,
    message="User updated settings",
    app_name="console_demo",
    user_id="user_bob_456",
    tenant_id="tenant_startup_inc",
    action="update_settings"
)
await logger.write(entry_user)


---
## Part 6: Real-World Example - API Endpoint Monitoring


In [ ]:
print("=" * 70)
print("Real-World Example: API Endpoint Monitoring")
print("=" * 70)

async def simulate_api_endpoint(endpoint, method, user_id):
    """Simulate an API endpoint and log the request."""
    import time
    start = time.time()

    # Simulate processing
    await asyncio.sleep(0.1)

    duration = int((time.time() - start) * 1000)

    entry = LogEntry.create(
        level=LogLevel.INFO,
        message=f"{method} {endpoint} - Request processed",
        app_name="console_json_demo",
        http_details={
            "method": method,
            "url": endpoint,
            "status_code": 200
        },
        user_id=user_id,
        tenant_id=f"tenant_{user_id.split('_')[1]}",
        duration_ms=duration,
        action="api_request"
    )
    await json_logger.write(entry)

# Simulate several API calls
print("\nSimulating API Requests:\n")

await simulate_api_endpoint("/api/projects", "GET", "user_alice_123")
await simulate_api_endpoint("/api/projects/123", "PUT", "user_alice_123")
await simulate_api_endpoint("/api/users/456", "GET", "user_bob_456")


---
## Summary


1. **ConsoleHandler** - Output logs to console in real-time
2. **Console_ServiceConfig** - Configure console output
3. **Output Types** - JSON for detailed logs, Text for simple messages
4. **Logger Setup** - Create logger with console handler
5. **Log Levels** - DEBUG, INFO, WARNING, ERROR, CRITICAL
6. **Rich Context** - HTTP details, entities, multi-tenant info
7. **Real-World Usage** - API endpoint monitoring

### Key Takeaways

- **Development Tool** - Perfect for local development and debugging
- **Two Formats** - JSON for rich logs, Text for simple output
- **No Configuration** - Works out of the box
- **Real-Time** - Immediate feedback
- **Rich Context** - All log details visible
